In [59]:
%%bash

pip install pandas==0.23.0
pip install numpy==1.14.3
pip install matplotlib==3.0.3
pip install seaborn==0.8.1
pip install PyAthena==1.8.0

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [60]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy.sparse import lil_matrix

import boto3
import botocore
import sagemaker

In [61]:
session = boto3.session.Session()
region_name = session.region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

In [62]:
from pyathena import connect
from pyathena.util import as_pandas

database_name = 'dsoaws'
table_name = 'amazon_reviews_parquet'
sentiment_table_name = 'amazon_reviews_with_sentiment'

s3_staging_dir = 's3://{0}/staging/athena'.format(bucket)

In [63]:
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()

cursor.execute('SELECT customer_id, \
                         review_id, \
                         product_id, \
                         product_title, \
                         review_headline, \
                         review_body, \
                         review_date, \
                         year, \
                         star_rating, \
                         sentiment, \
                         product_category \
                FROM {}.{} \
                WHERE product_category = \'Digital_Video_Download\' \
                ORDER BY review_date \
                LIMIT 500'
               .format(database_name, sentiment_table_name))
df = as_pandas(cursor)

In [64]:
df = df[['customer_id', 'product_id', 'product_title', 'star_rating', 'review_date']]

In [65]:
df.head(5)

,customer_id,product_id,product_title,star_rating,review_date
0,48685922,B000ICXQVA,Halloween III: Season of the Witch,2,11234
1,45764909,B001II0M9W,Simply Irresistible,3,11485
2,45764909,B00QFNLJZQ,A.I. Artificial Intelligence,4,11755
3,45764909,B00QFNLJZQ,A.I. Artificial Intelligence,4,11755
4,45764909,B009941RI2,Wuthering Heights,5,11781


In [66]:
df.shape

(500, 5)

In [67]:
customers = df['customer_id'].value_counts()
products = df['product_id'].value_counts()

quantiles = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99, 1]
print('customers\n', customers.quantile(quantiles))
print('products\n', products.quantile(quantiles))

customers
 0.00     1.0
0.01     1.0
0.02     1.0
0.03     1.0
0.04     1.0
0.05     1.0
0.10     1.0
0.25     1.0
0.50     1.0
0.75     2.0
0.90     2.0
0.95     3.0
0.96     3.0
0.97     4.0
0.98     5.0
0.99     7.0
1.00    13.0
Name: customer_id, dtype: float64
products
 0.00     1.00
0.01     1.00
0.02     1.00
0.03     1.00
0.04     1.00
0.05     1.00
0.10     1.00
0.25     1.00
0.50     1.00
0.75     2.00
0.90     2.00
0.95     4.00
0.96     4.00
0.97     4.00
0.98     6.00
0.99     7.92
1.00    14.00
Name: product_id, dtype: float64


In [68]:
num_customers = customers.count()
print(num_customers)
num_products = products.count()
print(num_products)
num_features = num_customers + num_products
print(num_features)

328
309
637


In [69]:
#customers = customers[customers >= 5]
#products = products[products >= 10]

reduced_df = df.merge(pd.DataFrame({'customer_id': customers.index})).merge(pd.DataFrame({'product_id': products.index}))

Create a sequential index for customers and movies

In [70]:
customers = reduced_df['customer_id'].value_counts()
products = reduced_df['product_id'].value_counts()

#customers = df['customer_id'].value_counts()
#products = df['product_id'].value_counts()

print(customers)
print(products)

48652051    13
47881267     9
16336396     8
17225887     7
45764909     7
18147129     5
37513072     5
53073229     5
53070836     5
15957545     4
16342208     4
52824002     4
52558011     4
17968658     3
22119842     3
52044147     3
46910749     3
51302967     3
34885907     3
52926333     2
18436735     2
15286599     2
9890951      2
52843621     2
30872364     2
44033364     2
37040213     2
27032484     2
43198240     2
30288577     2
            ..
51212387     1
43296246     1
13106503     1
37489005     1
24312762     1
40115248     1
47339197     1
40261920     1
11606284     1
15342724     1
52200874     1
14992988     1
50077390     1
16733506     1
48206751     1
52323087     1
43165337     1
30023223     1
29498914     1
47022220     1
34905157     1
53093634     1
48051637     1
31041536     1
32192325     1
50996592     1
41484249     1
49438424     1
52802983     1
50845031     1
Name: customer_id, Length: 328, dtype: int64
B000GT7XXW    14
B000MR1V22    10
B000NH

In [71]:
customer_index = pd.DataFrame({'customer_id': customers.index, 
                               'user': np.arange(customers.shape[0])})
print(customer_index)

product_index = pd.DataFrame({'product_id': products.index, 
                              'item': np.arange(products.shape[0]) + customer_index.shape[0]})
print(product_index)

    customer_id  user
0      48652051     0
1      47881267     1
2      16336396     2
3      17225887     3
4      45764909     4
5      18147129     5
6      37513072     6
7      53073229     7
8      53070836     8
9      15957545     9
10     16342208    10
11     52824002    11
12     52558011    12
13     17968658    13
14     22119842    14
15     52044147    15
16     46910749    16
17     51302967    17
18     34885907    18
19     52926333    19
20     18436735    20
21     15286599    21
22      9890951    22
23     52843621    23
24     30872364    24
25     44033364    25
26     37040213    26
27     27032484    27
28     43198240    28
29     30288577    29
..          ...   ...
298    51212387   298
299    43296246   299
300    13106503   300
301    37489005   301
302    24312762   302
303    40115248   303
304    47339197   304
305    40261920   305
306    11606284   306
307    15342724   307
308    52200874   308
309    14992988   309
310    50077390   310
311    167

---

## Train

- Create a [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) estimator to run a training jobs and specify:
  - Algorithm container image
  - IAM role
  - Hardware setup
  - S3 output location
  - Algorithm hyperparameters
    - `feature_dim`: $N + M + 1$ (additional feature is `days_since_first` to capture trend)
    - `num_factors`: number of factor dimensions (increasing too much can lead to overfitting)
    - `epochs`: number of full passes through the dataset
- `.fit()` points to training and test data in S3 and begins the training job

**Note**: For AWS accounts registered in conjunction with a workshop, default instance limits may prevent the use of `ml.c5.2xlarge` (and other equally powerful instances), and may require a lower value for `train_instance_count` depending on the instance type chosen. 

In [72]:
role = sagemaker.get_execution_role()

fm_prefix = 'factorization-machines'

base_job_name = 'factorization-machines'

fm = sagemaker.estimator.Estimator(
    sagemaker.amazon.amazon_estimator.get_image_uri(boto3.Session().region_name, 'factorization-machines', 'latest'),
    role, 
    train_instance_count=1,
    train_instance_type='ml.c5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, fm_prefix),
    base_job_name=base_job_name,
    sagemaker_session=sagemaker_session)

fm.set_hyperparameters(
    feature_dim=customer_index.shape[0] + product_index.shape[0] + 1,
    predictor_type='regressor',
    mini_batch_size=1000,
    num_factors=256,
    epochs=3)

fm.fit({'train': sagemaker.s3_input('s3://{}/{}/train/'.format(bucket, fm_prefix), distribution='ShardedByS3Key'), 
        'test': sagemaker.s3_input('s3://{}/{}/test/'.format(bucket, fm_prefix), distribution='FullyReplicated')},
        wait=False)

In [73]:
training_job_name = fm.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

training_job_name:  factorization-machines-2020-02-13-02-38-03-444


After some time, or in a separate Python notebook, we can attach to the running job using the `training_job_name`.

In [74]:
from sagemaker.estimator import Estimator

fm = Estimator.attach(training_job_name=training_job_name)

2020-02-13 02:38:03 Starting - Starting the training job...
2020-02-13 02:38:05 Starting - Launching requested ML instances.........
2020-02-13 02:39:39 Starting - Preparing the instances for training...
2020-02-13 02:40:31 Downloading - Downloading input data......
2020-02-13 02:41:31 Training - Training image download completed. Training in progress.
2020-02-13 02:41:31 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
/opt/amazon/lib/python2.7/site-packages/pandas/util/nosetester.py:13: DeprecationWarning: Importing from numpy.testing.nosetester is deprecated, import from numpy.testing instead.
  from numpy.testing import nosetester
[02/13/2020 02:41:28 INFO 140211299915584] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'factors_init_

---

## Host

Deploy trained model to a real-time production endpoint

In [75]:
fm_predictor = fm.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1)

---------------------!

Setup predictor to serialize in-memory data for invocation requests

In [76]:
import json

def fm_serializer(df):
    feature_dim = customer_index.shape[0] + product_index.shape[0] + 1
    js = {'instances': []}
    for index, data in df.iterrows():
        js['instances'].append({'data': {'features': {'values': [1, 1, data['days_since_first']],
                                                      'keys': [data['user'], data['item'], feature_dim - 1],
                                                      'shape': [feature_dim]}}})
    return json.dumps(js)

In [77]:
from sagemaker.predictor import json_deserializer

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

**Real-time prediction for what a single user would rate an item**

1. Pick a customer-movie pair from the dataset

In [78]:
test_df.head(10)

NameError: name 'test_df' is not defined

2. Pull out a single customer-product pair that we like

In [ ]:
test_customer = test_df.iloc[[0]]
test_customer = test_customer[['customer_id', 'user', 'item', 'days_since_first']]

test_customer.info()

3. Pass `test_customer` to predictor

In [ ]:
fm_predictor.predict(test_customer)

In [79]:
#fake_customer = test_customer 
desired_customer_id = '45764909' # test_customer['customer_id']
desired_user_id = 4 # int(test_customer['user'])
desired_item_id = 508 # int(test_customer['item'])
desired_review_days = 0

type(desired_review_days)
fake_customer_data = {'customer_id': desired_customer_id, 
                      'user' : desired_user_id, 
                      'item' : desired_item_id, 
                      'days_since_first' : desired_review_days}
fake_customer = pd.DataFrame(fake_customer_data, index=[0])
#fake_customer['customer_id'] = desired_customer_id
#fake_customer['user'] = desired_user_id
#fake_customer['item'] = desired_item_id
#fake_customer['days_since_first'] = desired_review_days

fake_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 4 columns):
customer_id         1 non-null object
user                1 non-null int64
item                1 non-null int64
days_since_first    1 non-null int64
dtypes: int64(3), object(1)
memory usage: 40.0+ bytes


In [95]:
#fake_customer = test_customer 
desired_customer_id = '10653514' # test_customer['customer_id']
desired_user_id = 240 # int(test_customer['user'])
desired_item_id = 451 # int(test_customer['item'])
desired_review_days = 0

type(desired_review_days)
fake_customer_data = {'customer_id': desired_customer_id, 
                      'user' : desired_user_id, 
                      'item' : desired_item_id, 
                      'days_since_first' : desired_review_days}
fake_customer = pd.DataFrame(fake_customer_data, index=[0])
#fake_customer['customer_id'] = desired_customer_id
#fake_customer['user'] = desired_user_id
#fake_customer['item'] = desired_item_id
#fake_customer['days_since_first'] = desired_review_days

fake_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 4 columns):
customer_id         1 non-null object
user                1 non-null int64
item                1 non-null int64
days_since_first    1 non-null int64
dtypes: int64(3), object(1)
memory usage: 40.0+ bytes


In [96]:
fm_predictor.predict(fake_customer)

{'predictions': [{'score': 0.30700334906578064}]}

## Clean-up

In [ ]:
# fm_predictor.delete_endpoint()